In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder

import matplotlib.colors as colors
import matplotlib.cbook as cbook
import matplotlib.pyplot as plt

import mlflow
import mlflow.tensorflow

import uuid

import glob

from IPython.display import display

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.metrics import multilabel_confusion_matrix

from eis.EISDataIO import eis_dataframe_from_csv

from os import path
import os
import logging

ModuleNotFoundError: No module named 'mlflow'

In [46]:
# loading training data

# if you are on a windows machine un-comment the following line to get the path to training data
# here = !echo %cd%

# if you are on a mac/ unix machine un-comment the following line to get the path to training data
here = !pwd

train_data_path = path.join(path.dirname(here[0]), "train_data.csv")
eis_data = eis_dataframe_from_csv(train_data_path)

In [51]:
#Add image path
image_folders_path = path.join(path.dirname(here[0]),"images/")

eis_data["image_name"] = image_folders_path + eis_data["Circuit"] + "/" + eis_data.index.astype(str)+".jpg"
eis_data.head()

,freq,Z,Circuit,Parameters,image_name
0,"[0.1, 0.148398179, 0.220220195, 0.326802759, 0...","[(309.82561192-156.06088233j), (267.46983919-1...",L-R-RCPE-RCPE-RCPE,"L1: 2.94e-08, R1: 4.51e+00, R2: 5.19e-02, CPE1...",/home/eis-user/images/L-R-RCPE-RCPE-RCPE/0.jpg
1,"[1.0, 1.34339933, 1.80472177, 2.42446202, 3.25...","[(344.50700012-0.87321496j), (344.36191597-0.9...",RC-RC-RCPE-RCPE,"R1: 2.08e+02, R2: 2.50e+01, R3: 9.57e+01, R4: ...",/home/eis-user/images/RC-RC-RCPE-RCPE/1.jpg
2,"[1.0, 1.26360956, 1.59670912, 2.01761691, 2.54...","[(3080.15920083-80.84202473j), (3071.83539583-...",L-R-RCPE-RCPE-RCPE,"L1: 3.35e-08, R1: 6.95e+01, R2: 7.49e+01, CPE1...",/home/eis-user/images/L-R-RCPE-RCPE-RCPE/2.jpg
3,"[10.0, 13.4990445, 18.2224203, 24.5985264, 33....","[(930.93345951-0.0068507146j), (930.93327153-0...",L-R-RCPE,"L1: 8.43e-07, R1: 9.06e+01, R2: 8.40e+02, CPE1...",/home/eis-user/images/L-R-RCPE/3.jpg
4,"[0.01, 0.0148907532, 0.0221734532, 0.033017942...","[(405.07355219-0.0149508921j), (405.07110253-0...",RCPE-RCPE-RCPE,"R1: 1.03e+01, R2: 6.71e-01, R3: 3.94e+02, CPE1...",/home/eis-user/images/RCPE-RCPE-RCPE/4.jpg


In [48]:
# Create base model
def create_cnn_model(num_classes: int,initial_lr:float = 0.01):
    inputs = layers.Input(shape=(224, 224, 3))
    base_model = keras.applications.MobileNetV2(input_shape=(224,224,3),input_tensor=inputs,weights="imagenet",include_top=False)

    # Freeze the pretrained weights
    base_model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(base_model.output)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(16,activation="relu",kernel_regularizer='l2')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(16,activation="relu")(x)

    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="MobileNetV2")
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model


In [49]:
model = create_cnn_model(num_classes=9,initial_lr=0.01)
model.summary()

Model: "MobileNetV2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                       

In [52]:
# Prepare Data generator

#Try zoom range?

datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255.,
                                                       zoom_range = 0.1,
                                                       validation_split=0.3)



train_gen = datagen.flow_from_dataframe(dataframe = eis_data,
                                        x_col="image_name",
                                        y_col="Circuit",
                                        subset="training",
                                        batch_size=32,
                                        shuffle=True,
                                        seed=42,
                                        target_size=(224,224),
                                        class_mode="categorical"
                                        )
val_gen = datagen.flow_from_dataframe(dataframe = eis_data,
                                        x_col="image_name",
                                        y_col="Circuit",
                                        subset="validation",
                                        batch_size=32,
                                        shuffle=False,
                                        seed=42,
                                        target_size=(224,224),
                                        class_mode="categorical"
                                        )

Found 5224 validated image filenames belonging to 9 classes.
Found 2238 validated image filenames belonging to 9 classes.


In [ ]:
# Enable auto-logging to MLflow to capture TensorBoard metrics.
mlflow.tensorflow.autolog()

In [2]:
# Train model
with mlflow.start_run():
    # Set checkpoint for best model
    model_name = str(uuid.uuid4())
    checkpoint_filepath =  path.join(path.dirname(here[0]), "models",f"{model_name}.h5")
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=False,
        monitor='val_accuracy',
        mode='auto',
        save_best_only=True)
    
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001,patience=60),
                 keras.callbacks.ReduceLROnPlateau(factor=0.01,min_delta= 0.001, patience=15),
                 model_checkpoint_callback]

    step_size_train = train_gen.n//train_gen.batch_size
    step_size_val = val_gen.n//val_gen.batch_size

    h = model.fit_generator(generator = train_gen,
                    steps_per_epoch=step_size_train,
                    validation_data=val_gen,
                    validation_steps=step_size_val,
                    epochs=1000,
                    callbacks=callbacks)

NameError: name 'mlflow' is not defined